<a href="https://colab.research.google.com/github/Nagmashaik123/Nagma_INFO5731_Spring2021/blob/main/In_class_exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (20 points in total, 2/9/2021)**

# 1. Text Data Preprocessing

Here is a [legal case](https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



## 1.1 Basic feature extraction using text data (4 points)

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

## 1.2 Basic Text Pre-processing of text data (4 points)

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

## 1.3 Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above. (4 points)


## 1.4 Advance Text Processing (Extra credit: 4 points)

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.


1.1  Basic feature extraction using text data

In [241]:
# Program on text data preprocessing
import pandas as pd
from six.moves import urllib
import nltk
nltk.download('stopwords')


array_data = []
url_to_read = "https://raw.githubusercontent.com/unt-iialab/info5731_spring2021/main/class_exercises/01-05-1%20%20Adams%20v%20Tanner.txt"
file_to_read = urllib.request.urlopen(url_to_read)

for item in file_to_read:
  modified_data = item.decode("utf-8").replace('\r\n', '')
  array_data.append(modified_data)

while("" in array_data) : 
    array_data.remove("") 
    
df = pd.DataFrame (array_data, columns = ['original_text'])


# using pandas representation for columns
df['review_sentence_count'] = df['original_text'].apply(lambda original_text:len(str(original_text).split(".")))
df['review_words_count'] = df['original_text'].apply(lambda original_text: len(str(original_text).split(" ")))
df['review_char_count'] = df['original_text'].str.len()

# calculation of word length.
def avg_wrd_len_cal(column_statement):
  words_per_line = column_statement.split()
  if len(words_per_line) != 0:
    return(sum(len(word) for word in words_per_line)/len(words_per_line))

# avg word length column
df['review_avg_word_len'] = df['original_text'].apply(lambda original_text: avg_wrd_len_cal(original_text))

stop = stopwords.words('english')

# stop words colum
df['review_stop_words_count'] = df['original_text'].apply(lambda original_text: len([original_text for original_text in original_text.split() if original_text in stop]))
df

# no of special characters for each column
def special_chars(col_stat):
  chars = 0
  for item in range(len(col_stat)):
    if not((col_stat[item].isalpha()) and (col_stat[item].isdigit())):
        chars = chars + 1
  return chars

df['review_special_char'] = df['original_text'].apply(lambda original_text: special_chars(original_text))

# no of numerics 
df['review_no_numerics'] = df['original_text'].apply(lambda original_text: len([original_text for original_text in original_text.split() if original_text.isdigit()]))

# no of upper case chars
df['review_no_upper_words'] = df['original_text'].apply(lambda original_text: len([original_text for original_text in original_text.split() if original_text.isupper()]))

df


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,original_text,review_sentence_count,review_words_count,review_char_count,review_avg_word_len,review_stop_words_count,review_special_char,review_no_numerics,review_no_upper_words
0,5 Ala. 740,2,3,10,2.666667,0,10,2,0
1,Supreme Court of Alabama.,2,4,25,5.500000,1,25,0,0
2,ADAMS,1,1,5,5.000000,0,5,0,1
3,v.,2,1,2,2.000000,0,2,0,0
4,TANNER AND HORTON.,2,3,18,5.333333,0,18,0,3
...,...,...,...,...,...,...,...,...,...
143,There are no Filings for this citation.,2,7,39,4.714286,4,39,0,0
144,Negative Treatment,1,2,18,8.500000,0,18,0,0
145,There are no Negative Treatment results for th...,2,9,58,5.555556,4,58,0,0
146,History,1,1,7,7.000000,0,7,0,0


In [243]:
# Returning the count of sentences , words and characters

sentence_count = df['review_sentence_count'].sum()
word_count = df['review_words_count'].sum()
char_count = df['review_char_count'].sum()
word_len = df['review_avg_word_len'].sum()
stop_words = df['review_stop_words_count'].sum()
special_chars = df['review_special_char'].sum()
no_numerics = df['review_no_numerics'].sum()
no_upper_words = df['review_no_upper_words'].sum()



print("Total no sentences : " , sentence_count) 
print("Total no words : " , word_count)
print("Total no characters : " , char_count)
print("Total avg word length : " , word_len)
print("Total no of stop words : " , stop_words)
print("Total no of special chars : " , special_chars)
print("Total no of numerics : " , no_numerics)
print("Total no of upper words : ", no_upper_words)



Total no sentences :  439
Total no words :  3722
Total no characters :  20295
Total avg word length :  642.1036748280112
Total no of stop words :  1679
Total no of special chars :  20295
Total no of numerics :  60
Total no of upper words :  84


1.2 Basic Text Pre-processing of text data

In [244]:

import nltk
from textblob import Word
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('wordnet')
stop = stopwords.words('english')

df['convert_lower_case'] = df['original_text'].apply(lambda original_text: " ".join(original_text.lower() for original_text in original_text.split()))
df['punctuation_removal'] = df['original_text'].str.replace('[^\w\s]','')
df['Stopwords_removal'] = df['punctuation_removal'].apply(lambda original_text: " ".join(original_text for original_text in original_text.split() if original_text not in stop))

dup_words = pd.Series(' '.join(df['Stopwords_removal']).split()).value_counts()[:10]
print("Duplicate words to be removed : \n",dup_words)
dup_words_list = list(dup_words.index)

# removing frequency words
df['frequent_words_removal'] = df['Stopwords_removal'].apply(lambda original_text: " ".join(original_text for original_text in original_text.split() if original_text not in dup_words_list))

# removing rare words 
rare_words = pd.Series(' '.join(df['Stopwords_removal']).split()).value_counts()[-10:]
print("Rare words to be removed : \n",rare_words)
rare_words_list = list(rare_words.index)
df['Rare_words_removal'] = df['frequent_words_removal'].apply(lambda original_text: " ".join(original_text for original_text in original_text.split() if original_text not in rare_words_list))

# Special correction
df['Spelling_correction'] = df['Rare_words_removal'].apply(lambda original_text: str(TextBlob(original_text).correct()))

# Tokenization 
df['Tokenization'] = df['Spelling_correction'].apply(lambda original_text: TextBlob(original_text).words)

# Stemming
st = PorterStemmer()
df['Stemming'] = df['Tokenization'].apply(lambda original_text: " ".join([st.stem(word) for word in original_text]))

df['Lemmatization'] = df['Stemming'].apply(lambda original_text: " ".join([Word(word).lemmatize() for word in original_text.split()]))
df



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Duplicate words to be removed : 
 execution    50
crop         49
The          28
levy         25
lien         24
claimants    22
v            22
right        20
gathered     19
contract     16
dtype: int64
Rare words to be removed : 
 temporary        1
1857             1
108              1
retrospective    1
cultivated       1
1828             1
bona             1
Clays            1
Attorneys        1
West             1
dtype: int64


,original_text,review_sentence_count,review_words_count,review_char_count,review_avg_word_len,review_stop_words_count,review_special_char,review_no_numerics,review_no_upper_words,convert_lower_case,punctuation_removal,Stopwords_removal,frequent_words_removal,Rare_words_removal,Spelling_correction,Tokenization,Stemming,Lemmatization
0,5 Ala. 740,2,3,10,2.666667,0,10,2,0,5 ala. 740,5 Ala 740,5 Ala 740,5 Ala 740,5 Ala 740,5 La 740,"[5, La, 740]",5 La 740,5 La 740
1,Supreme Court of Alabama.,2,4,25,5.500000,1,25,0,0,supreme court of alabama.,Supreme Court of Alabama,Supreme Court Alabama,Supreme Court Alabama,Supreme Court Alabama,Supreme Court Alabama,"[Supreme, Court, Alabama]",suprem court alabama,suprem court alabama
2,ADAMS,1,1,5,5.000000,0,5,0,1,adams,ADAMS,ADAMS,ADAMS,ADAMS,ADAMS,[ADAMS],adam,adam
3,v.,2,1,2,2.000000,0,2,0,0,v.,v,v,,,,[],,
4,TANNER AND HORTON.,2,3,18,5.333333,0,18,0,3,tanner and horton.,TANNER AND HORTON,TANNER AND HORTON,TANNER AND HORTON,TANNER AND HORTON,TANNER AND HORTON,"[TANNER, AND, HORTON]",tanner and horton,tanner and horton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,There are no Filings for this citation.,2,7,39,4.714286,4,39,0,0,there are no filings for this citation.,There are no Filings for this citation,There Filings citation,There Filings citation,There Filings citation,There Tidings situation,"[There, Tidings, situation]",there tide situat,there tide situat
144,Negative Treatment,1,2,18,8.500000,0,18,0,0,negative treatment,Negative Treatment,Negative Treatment,Negative Treatment,Negative Treatment,Negative Treatment,"[Negative, Treatment]",neg treatment,neg treatment
145,There are no Negative Treatment results for th...,2,9,58,5.555556,4,58,0,0,there are no negative treatment results for th...,There are no Negative Treatment results for th...,There Negative Treatment results citation,There Negative Treatment results citation,There Negative Treatment results citation,There Negative Treatment results situation,"[There, Negative, Treatment, results, situation]",there neg treatment result situat,there neg treatment result situat
146,History,1,1,7,7.000000,0,7,0,0,history,History,History,History,History,History,[History],histori,histori


In [245]:
# Saving data to CSV file
df.to_csv('final_data.csv',index=False)

In [246]:
#1.4 Advance Text Processing

tf1 = (df['Lemmatization']).apply(lambda original_text: pd.value_counts(original_text.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words_list','term_frequency']
tf1

,words_list,term_frequency
0,5,8.0
1,740,2.0
2,La,15.0
3,court,20.0
4,alabama,1.0
...,...,...
725,yield,1.0
726,rye,1.0
727,tide,2.0
728,neg,2.0


In [247]:
# Top 10 - one grams
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

word_array = []
for item in df['Lemmatization']:
  word_array.append(word_tokenize(item))
dataProcessed = [x for x in word_array if x != []]
cycleData = list(itertools.chain.from_iterable(dataProcessed))
oneGrams = ngrams(cycleData, 1)
collections.Counter(oneGrams).most_common(10)


[(('case',), 24),
 (('court',), 20),
 (('grow',), 17),
 (('law',), 17),
 (('rep',), 16),
 (('La',), 15),
 (('plaintiff',), 15),
 (('posse',), 14),
 (('attach',), 14),
 (('cotton',), 14)]

# 2. Python Regular Expression

## 2.1 Write a Python program to remove leading zeros from an IP address. (4 points)

ip = "260.08.094.109"

In [248]:
# Progran to remove leading zeros from IP address
import re
ip =  "260.08.094.109"
address_with_out_zero = re.sub('\.[0]*', '.', ip)
print(address_with_out_zero)






260.8.94.109


## 2.2 Write a Python Program to extract all the years from the following sentence. (4 points)

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [249]:
# Program to extract all the years
import re
sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."
years_extracted = re.findall(r'2\d\d\d', sentence)
print(years_extracted)

['2010', '2010', '2019']
